In [72]:
import os
import itertools
import xlsxwriter

import numpy as np
import pandas as pd
from scipy.optimize import minimize
from sklearn.model_selection import TimeSeriesSplit
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_validate
import matplotlib.pyplot as plt

import DataPreparation as dpr
import ModelRun as mr
import benchmarks as bench
from MyEstimators import CLS_Estimator

### Load data

In [73]:
df = dpr.read_data('EQP_Quarterly')
df = dpr.data_clean(df, '1956-01-01')

In [74]:
df.head()

,EQP,DP,DY,EP,DE,svar,b/m,ntis,tbl,lty,...,TMS,DFR,DFY,infl,c,w,y,cay,AAA,BAA
time,,,,,,,,,,,,,,,,,,,,,
1956-03-01,0.066512,-3.333030,-3.269151,-2.575525,-0.757505,0.003289,0.531077,0.026695,0.0225,0.0303,...,0.0078,0.005159,0.0050,0.000000,9.272498,11.092725,9.100386,0.007275,0.0310,0.0360
1956-06-01,-0.028264,-3.261722,-3.293365,-2.568575,-0.693147,0.003688,0.551565,0.025672,0.0249,0.0299,...,0.0050,-0.021824,0.0050,0.014925,9.271728,11.091665,9.107828,0.000775,0.0326,0.0376
1956-09-01,-0.034415,-3.204645,-3.239744,-2.573142,-0.631503,0.002519,0.571910,0.029362,0.0284,0.0324,...,0.0040,0.005663,0.0051,0.007353,9.269304,11.086198,9.106428,0.000663,0.0356,0.0407
1956-12-01,0.033241,-3.289216,-3.260525,-2.616389,-0.672827,0.004394,0.544177,0.026149,0.0321,0.0345,...,0.0024,-0.002208,0.0062,0.007299,9.277993,11.096678,9.118405,-0.002524,0.0375,0.0437
1957-03-01,-0.050750,-3.238565,-3.294980,-2.562911,-0.675654,0.002288,0.599819,0.026600,0.0308,0.0331,...,0.0023,-0.000368,0.0077,0.007246,9.280482,11.090721,9.117433,0.002041,0.0366,0.0443


### Add $y_{t-1}$ and construct X and y

In [75]:
df['y_lag'] = df['EQP'].shift()
df = df.dropna()
df.head()

,EQP,DP,DY,EP,DE,svar,b/m,ntis,tbl,lty,...,DFR,DFY,infl,c,w,y,cay,AAA,BAA,y_lag
time,,,,,,,,,,,,,,,,,,,,,
1956-06-01,-0.028264,-3.261722,-3.293365,-2.568575,-0.693147,0.003688,0.551565,0.025672,0.0249,0.0299,...,-0.021824,0.0050,0.014925,9.271728,11.091665,9.107828,0.000775,0.0326,0.0376,0.066512
1956-09-01,-0.034415,-3.204645,-3.239744,-2.573142,-0.631503,0.002519,0.571910,0.029362,0.0284,0.0324,...,0.005663,0.0051,0.007353,9.269304,11.086198,9.106428,0.000663,0.0356,0.0407,-0.028264
1956-12-01,0.033241,-3.289216,-3.260525,-2.616389,-0.672827,0.004394,0.544177,0.026149,0.0321,0.0345,...,-0.002208,0.0062,0.007299,9.277993,11.096678,9.118405,-0.002524,0.0375,0.0437,-0.034415
1957-03-01,-0.050750,-3.238565,-3.294980,-2.562911,-0.675654,0.002288,0.599819,0.026600,0.0308,0.0331,...,-0.000368,0.0077,0.007246,9.280482,11.090721,9.117433,0.002041,0.0366,0.0443,0.033241
1957-06-01,0.075114,-3.309868,-3.238565,-2.628349,-0.681519,0.001363,0.565877,0.030528,0.0329,0.0361,...,-0.003789,0.0072,0.010791,9.278119,11.104916,9.118823,-0.004528,0.0391,0.0463,-0.050750


In [76]:
X = df[['DP','DY']]
station = pd.DataFrame()
y = df[['EQP']].squeeze()
X.head(2)

,DP,DY
time,,
1956-06-01,-3.261722,-3.293365
1956-09-01,-3.204645,-3.239744


### Construct single-index and nonlinear models

In [77]:
def single_index(x):
    if isinstance(x, (pd.DataFrame, np.ndarray)):
        if isinstance(x, pd.DataFrame):
            x_values = x.values
        else:
            pass
    else:
        raise Exception('wrong type')

    def u(theta):
        if len(theta) == x_values.shape[1]:
            sum_up = [x_values[:, i] * theta[i] for i in range(x_values.shape[1])]
            index = np.sum(sum_up, axis=0)
        else:
            raise Exception('wrong parameter dimension')
        return index

    return u

In [78]:
extra_params = {'sin_func':1,
               'cos_func':1,
               'scaled_sin_func':2,
               'scaled_cos_func':2,
               'exp_func':2,
               'exp_shift_func':2,
                'poly_func':3,
                'linear_func':2
               }

In [79]:
def dimensions(non_sta, sta, func):
    stas = sta.shape[1]
    nonstas = non_sta.shape[1]
    extra = range(0, extra_params[func])
    return nonstas, stas, extra

In [80]:
def sin_func(x):
    def objective_func(params):
        func = np.sin(single_index(x.iloc[:,:d1])(params[0:d1])+params[d1+d2+extra[0]])+np.dot(
            x.iloc[:,d1:d1+d2], params[d1:d1+d2])
        return func
    return objective_func

In [81]:
def cos_func(x):
    def objective_func(params):
        func = np.cos(single_index(x.iloc[:,:d1])(params[0:d1])+params[d1+d2+extra[0]])+np.dot(
            x.iloc[:,d1:d1+d2], params[d1:d1+d2])
        return func
    return objective_func

In [82]:
def scaled_sin_func(x):
    def objective_func(params):
        func = np.sin(params[d1+d2+extra[1]]*single_index(x.iloc[:,:d1])(
            params[0:d1])+params[d1+d2+extra[0]])+np.dot(x.iloc[:,d1:d1+d2], params[d1:d1+d2])
        return func
    return objective_func

In [83]:
def scaled_cos_func(x):
    def objective_func(params):
        func = np.cos(params[d1+d2+extra[1]]*single_index(x.iloc[:,:d1])(
            params[0:d1])+params[d1+d2+extra[0]])+np.dot(x.iloc[:,d1:d1+d2], params[d1:d1+d2])
        return func
    return objective_func

In [84]:
def exp_shift_func(x):
    def objective_func(params):
        func = 1 - np.exp(params[d1+d2+extra[1]]*((single_index(x.iloc[:,:d1])(
            params[0:d1]))-params[d1+d2+extra[0]])**2)+np.dot(x.iloc[:,d1:d1+d2], params[d1:d1+d2])
        return func
    return objective_func

In [85]:
def exp_func(x):
    def objective_func(params):
        func = params[d1+d2+extra[0]]*np.exp(-params[d1+d2+extra[1]]*(single_index(x.iloc[:,:d1])(params[0:d1]))**2
                                )+np.dot(x.iloc[:,d1:d1+d2], params[d1:d1+d2])
        return func
    return objective_func

In [86]:
def poly_func(x):
    def objective_func(params):
        func = params[d1+d2+extra[0]]+params[d1+d2+extra[1]]*(single_index(x.iloc[:,:d1])(
            params[0:d1]))+params[d1+d2+extra[2]]*((single_index(x.iloc[:,:d1])(
            params[0:d1]))**2)+np.dot(x.iloc[:,d1:d1+d2], params[d1:d1+d2])
#                (single_index(x.iloc[:,:d1])(params[0:d1])
        return func
    return objective_func

In [87]:
def linear_func(x):
    def objective_func(params):
        func = params[d1+d2+extra[0]]+params[d1+d2+extra[1]]*(single_index(x.iloc[:,:d1])(
            params[0:d1]))+np.dot(x.iloc[:,d1:d1+d2], params[d1:d1+d2])
#                (single_index(x.iloc[:,:d1])(params[0:d1])
        return func
    return objective_func

### Model Estimation

In [88]:
def constraint_func(x):
    def constraint(params):
        con = 0
        for j in np.arange(0, x.iloc[:,:d1].shape[1]):
            con += params[j]**2
            cons = con - 1
        return cons
    return {'type':'eq', 'fun': constraint}

In [150]:
station

""


In [149]:
d1, d2, extra = dimensions(X, station,'poly_func')
print(d1, d2, extra)

2 0 range(0, 3)


In [146]:
d1, d2, extra = dimensions(X, station,'poly_func')
cls_nls = CLS_Estimator(obj_func = poly_func, x0 = [0.001]*(d1+d2+extra[-1]+1))
cls = CLS_Estimator(obj_func = poly_func, x0 = [0.001]*(d1+d2+extra[-1]+1), constraints = constraint_func(X))

In [147]:
cls_nls.fit(X,y)
cls.fit(X,y)

CLS_Estimator(constraints={'fun': <function constraint_func.<locals>.constraint at 0x0000026F078F28C8>,
                           'type': 'eq'},
              obj_func=<function poly_func at 0x0000026F09AB4950>,
              x0=[0.001, 0.001, 0.001, 0.001, 0.001])

In [148]:
print(cls_nls.params_)
print(cls.params_)

[-8.94841330e-04  1.92702711e-03  1.24399499e-02 -2.56973660e-05
  1.01218548e-03]
[-0.16930924  0.98556298  0.00341686 -0.03364998 -0.01037493]


In [22]:
cls.params_[0]**2+cls.params_[1]**2

1.0000000010637105

## Empirical Study

### Cointegrated predictors
- dividend-price ratio and dividend yield
- T-bill rate and long-term yield
- dividend-price ratio and earningprice ratio
- baa- and aaa-rated corporate bond yields

In [170]:
co1 = df[['DP', 'DY']]
co2 = df[['tbl', 'lty']]
co3 = df[['DP', 'EP']]
co4 = df[['BAA', 'AAA']]

In [171]:
cointe_variables = [co1, co2, co3, co4]

### Stationary variables

In [172]:
station = df[['y_lag', 'cay']]

### Fit model and Save Results

In [173]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

In [174]:
fun_list = [sin_func,
            cos_func,
            scaled_sin_func,
            scaled_cos_func,
            exp_func,
            exp_shift_func,
            poly_func,
            linear_func
           ]

In [175]:
# Set up hierachical index
fun_names = [i.__name__ for i in fun_list]
cointe_names = [get_df_name(i) for i in cointe_variables]
iterables_a = [fun_names, cointe_names]

In [176]:
#Set up directory
parent = os.getcwd()
folder = 'results'
path = os.path.join(parent, folder)
if not os.path.exists(path):
    os.makedirs(path)

In [177]:
results = pd.DataFrame()
for i, j in itertools.product(fun_list, cointe_variables):
    # Set up dimensions
    d1, d2, extra= dimensions(j,station, i.__name__)
    initial_len = d1+d2+extra[-1]+1
    
    # Set up dataframes
    iterables = [[i.__name__], [get_df_name(j)]]
    sec_columns = ['param_'+str(i) for i in range(1,initial_len+1)]
    multi_index = pd.MultiIndex.from_product(iterables, names=["function", "variables"])
    multi_columns = pd.MultiIndex.from_product([['NLS', 'CLS'], sec_columns],
                                               names=['Estimator', 'Parameters'])
    result = pd.DataFrame(index = multi_index, columns = multi_columns)
    # Prepare X
    X_ = j.join(station)
    # Fit models
    nls = CLS_Estimator(obj_func = i, x0 = [0.001]*initial_len)
    cls = CLS_Estimator(obj_func = i, x0 = [0.001]*initial_len, constraints = constraint_func(X_))
    nls.params_ = nls.fit(X_,y).params_
    cls.params_ = cls.fit(X_,y).params_
    # Save results to dataframe
    result.loc[i.__name__,get_df_name(j)].loc['NLS'] = nls.params_ 
    result.loc[i.__name__,get_df_name(j)].loc['CLS'] = cls.params_ 
    # Put into one table
    results = results.append(result, ignore_index = False, sort = False)
    
# Export to Excel
results.to_excel('Results/full_sample.xlsx')

In [178]:
results.head(3)

Estimator                CLS                                                   \
Parameters           param_1   param_2   param_3   param_4    param_5 param_6   
function variables                                                              
sin_func co1        0.709838 -0.704365  0.022517 -0.104848 -15.689339     NaN   
         co2        0.495116 -0.868827  0.079209  0.054524 -50.222284     NaN   
         co3        0.715437 -0.698678 -0.221696 -0.790349  -2.565328     NaN   

Estimator                        NLS                                          \
Parameters         param_7   param_1   param_2   param_3   param_4   param_5   
function variables                                                             
sin_func co1           NaN -1.008436  1.002847  0.000839 -0.073964 -0.023462   
         co2           NaN -0.786401  0.690315  0.066833 -0.247875  0.003686   
         co3           NaN -0.026356  -0.00009  0.059006 -0.224006 -0.083221   

Estimator                           
Parameters         param_6 param_7  
function variables                  
sin_func co1           NaN     NaN  
         co2           NaN     NaN  
         co3           NaN     NaN

# WTF is hapenning here?!!!

In [179]:
# def fit_model(functions, variables, file_name):   
#     results = pd.DataFrame()
#     for i, j in itertools.product(functions, variables):
#         # Set up dimensions
#         d1, d2, extra= dimensions(j,station, i.__name__)
# #         print(d1,d2,extra[-1])
#         initial_len = d1+d2+extra[-1]+1

#         # Set up dataframes
#         iterables = [[i.__name__], [get_df_name(j)]]
#         sec_columns = ['param_'+str(i) for i in range(1,initial_len+1)]
#         multi_index = pd.MultiIndex.from_product(iterables, names=["function", "variables"])
#         multi_columns = pd.MultiIndex.from_product([['NLS', 'CLS'], sec_columns],
#                                                    names=['Estimator', 'Parameters'])
#         result = pd.DataFrame(index = multi_index, columns = multi_columns)
#         # Prepare X
#         X_ = j.join(station)
#         # Fit models
#         nls = CLS_Estimator(obj_func = i, x0 = [0.001]*initial_len)
#         cls = CLS_Estimator(obj_func = i, x0 = [0.001]*initial_len, constraints = constraint_func(X))
#         nls.params_ = nls.fit(X_,y).params_
#         cls.params_ = cls.fit(X_,y).params_
#         print(initial_len)
#         # Save results to dataframe
#         result.loc[i.__name__,get_df_name(j)].loc['NLS'] = nls.params_ 
#         result.loc[i.__name__,get_df_name(j)].loc['CLS'] = cls.params_ 
#         # Put into one table
#         results = results.append(result, ignore_index = False, sort = False)
#     # Export to Excel
#     path = 'Results/' + file_name +'.xlsx'
#     results.to_excel(path)
#     return results

In [180]:
# fit_model(fun_list, cointe_variables,'results_1835')

## Use initial values from Linear regression (using Taylor expansion)

In [181]:
orders = {'sin_func':1,
          'cos_func':2,
          'scaled_sin_func':1,
          'scaled_cos_func':2,
          'exp_func':5,
          'exp_shift_func':4,
          'poly_func':2,
          'linear_func':1
               }

In [182]:
def Taylor_init(variables, station, y, function):
    
    d1, d2, extra = dimensions(variables, station, function.__name__)
    
    # find the initials for theta
    LR = LinearRegression()
    LR_theta = LR.fit(variables.iloc[:,1:], variables.iloc[:,:1])
    alpha = np.append(1, -LR_theta.coef_)
    theta = np.array(-alpha/np.linalg.norm(alpha))
#     print(len(theta))
    
    # calculate single-index
    u = single_index(variables)(theta)
    
    # find the initials for beta
    Xs = station.copy()
#     print(Xs.shape[1])
    Xs['u'], Xs['u2'], Xs['u3'], Xs['u4'], Xs['u6'] = u, u**2, u**3, u**4, u**6
    t_order = orders.get(function.__name__)
    X_reg = Xs.iloc[:, 0:d2+t_order]
#     print(d2, t_order)
    LR_taylor = LR.fit(X_reg, y)
    theta_gamma = np.append(theta,LR_taylor.coef_[:d2])
#     print(len(theta_gamma))
    
    
    # initials for gammas
    initials = []
    if function == sin_func:
        initials = np.append(theta_gamma, LR_taylor.intercept_)
    elif function == scaled_sin_func:
        initials = np.append(theta_gamma, ([LR_taylor.intercept_], [LR_taylor.coef_[0]]))
    elif function == linear_func:
        initials = np.append(theta_gamma, ([LR_taylor.intercept_], [LR_taylor.coef_[0]]))
    elif function == poly_func:
        ini_poly_ = np.append(theta_gamma,LR_taylor.coef_[d2:])
        initials = np.insert(ini_poly_, 4, LR_taylor.intercept_)
    elif function == cos_func:
        initials = np.append(theta_gamma, [-LR_taylor.coef_[d2]])
    elif function == scaled_cos_func:
        initials = np.append(theta_gamma,(
            [-LR_taylor.coef_[2]/np.sqrt((1+LR_taylor.intercept_)*2)], [np.sqrt((1+LR_taylor.intercept_)*2)]))
    elif function == exp_shift_func:
        initials = np.append(theta_gamma, [LR_taylor.coef_[-2]/(2*LR_taylor.coef_[-1]), np.sqrt(np.abs(LR_taylor.coef_[-1]))])
#         print(LR_taylor.coef_, LR_taylor.coef_[-2])
    elif function == exp_func:
        initials =np.append(theta_gamma, [LR_taylor.intercept_, -LR_taylor.coef_[2]/LR_taylor.intercept_])

    return initials

# Fit model and Save Results

In [183]:
# Taylor_init(co1, station, y, sin_func)

In [184]:
results_Taylor = pd.DataFrame()
for i, j in itertools.product(fun_list, cointe_variables):
    # Set up dimensions
    d1, d2, extra= dimensions(j,station, i.__name__)
    initial_len = d1+d2+extra[-1]+1
    # Set up dataframes
    iterables = [[i.__name__], [get_df_name(j)]]
    sec_columns = ['param_'+str(i) for i in range(1,initial_len+1)]
    multi_index = pd.MultiIndex.from_product(iterables, names=["function", "variables"])
    multi_columns = pd.MultiIndex.from_product([['NLS', 'CLS'], sec_columns],
                                               names=['Estimator', 'Parameters'])
    result = pd.DataFrame(index = multi_index, columns = multi_columns)
    # Prepare X
    X_ = j.join(station)
    # Fit models
    
    nls = CLS_Estimator(obj_func = i, x0 = Taylor_init(j, station, y, i))
    cls = CLS_Estimator(obj_func = i, x0 = Taylor_init(j, station, y, i), constraints = constraint_func(X_))
    nls.params_ = nls.fit(X_,y).params_
    cls.params_ = cls.fit(X_,y).params_
    # Save results to dataframe
    result.loc[i.__name__,get_df_name(j)].loc['NLS'] = nls.params_ 
    result.loc[i.__name__,get_df_name(j)].loc['CLS'] = cls.params_ 
    # Put into one table
    results_Taylor = results_Taylor.append(result, ignore_index = False, sort = False)
    
# Export to Excel
results_Taylor.to_excel('Results/Taylor_fullsample_0709.xlsx')
results_Taylor.tail()

Estimator                   CLS                                          \
Parameters              param_1   param_2   param_3   param_4   param_5   
function    variables                                                     
poly_func   co4       -0.639086  0.769135  0.029136  0.222809  0.029647   
linear_func co1       -0.709067  0.705141  0.000527 -0.075851 -0.023234   
            co2       -0.751669  0.659541  0.066719 -0.248713  0.003689   
            co3       -0.999994 -0.003524  0.059013 -0.223993 -0.083197   
            co4        0.476652 -0.879092   0.07649 -0.090205  0.023524   

Estimator                                          NLS                      \
Parameters              param_6      param_7   param_1   param_2   param_3   
function    variables                                                        
poly_func   co4        0.171961 -1066.824933 -0.647757  0.777972  0.028445   
linear_func co1        1.416036          NaN -1.080529  1.074592  0.000564   
            co2         1.04774          NaN -0.857493  0.752473  0.066743   
            co3        0.026346          NaN  -0.78005 -0.003462  0.059018   
            co4        0.527701          NaN -1.472724 -0.020691  0.072875   

Estimator                                                         
Parameters              param_4   param_5   param_6      param_7  
function    variables                                             
poly_func   co4        0.229783  0.030041 -0.098865 -1066.754865  
linear_func co1       -0.075504 -0.023073  0.928931          NaN  
            co2       -0.249108  0.003693  0.919095          NaN  
            co3       -0.223942 -0.083234  0.033766          NaN  
            co4       -0.111111  0.024863  0.113793          NaN

### GridSearch and CrossValidation

### Train_test split

In [38]:
# val_length = 1
test_length = 31
step = 1
### quarterly data:4
freq = 4
# cv_outer = TimeSeriesSplit(gap=0, max_train_size=None, n_splits=int((12/step) * test_length), test_size=step)
# cv_inner = TimeSeriesSplit(gap=0, max_train_size=None, n_splits=int((12/step) * val_length), test_size=step)
cv_outer = TimeSeriesSplit(gap=0, max_train_size=None, n_splits=test_length*freq, test_size=step)
# cv_inner = TimeSeriesSplit(gap=0, max_train_size=None, n_splits=4, test_size=step)

In [190]:
df_AR2 = df.copy()
df_AR2['y_lag2'] = df_AR2['EQP'].shift()
df_AR2 = df_AR2.dropna()
df_AR2.head(2)

,EQP,DP,DY,EP,DE,svar,b/m,ntis,tbl,lty,...,DFY,infl,c,w,y,cay,AAA,BAA,y_lag,y_lag2
time,,,,,,,,,,,,,,,,,,,,,
1956-09-01,-0.034415,-3.204645,-3.239744,-2.573142,-0.631503,0.002519,0.571910,0.029362,0.0284,0.0324,...,0.0051,0.007353,9.269304,11.086198,9.106428,0.000663,0.0356,0.0407,-0.028264,-0.028264
1956-12-01,0.033241,-3.289216,-3.260525,-2.616389,-0.672827,0.004394,0.544177,0.026149,0.0321,0.0345,...,0.0062,0.007299,9.277993,11.096678,9.118405,-0.002524,0.0375,0.0437,-0.034415,-0.034415


In [236]:
co1 = df_AR2[['DP', 'DY']]
co2 = df_AR2[['tbl', 'lty']]
co3 = df_AR2[['DP', 'EP']]
co4 = df_AR2[['BAA', 'AAA']]
station_AR2 = df_AR2[['y_lag', 'cay']]

In [237]:
y_lag2 = df_AR2[['EQP']].squeeze()

In [238]:
X_train_AR2 = df_AR2.loc[:"1988-01-01"]
y_train_AR2 = y_lag2.loc[:"1988-01-01"]

X_test_AR2 = df_AR2.loc["1988-01-01":"2018-12-01"]
y_test_AR2 = y_lag2.loc["1988-01-01":"2018-12-01"]
station_train = df_AR2.loc[:"1988-01-01"][['y_lag', 'cay']]

In [239]:
variables_lag2 = [co1, co2, co3, co4]

In [338]:
fun_list = [sin_func, cos_func]

In [424]:
oos_R2 = pd.DataFrame()
oos_MSE = pd.DataFrame()

rows = df.loc["1988-01-01":"2018-12-01"].index
sec_columns = ['CLS_MSE', 'SM_MSE', 'AR1_MSE', 'AR2_MSE', 'AR_cay_MSE']
multi_columns = pd.MultiIndex.from_product([['co1', 'co2', 'co3', 'co4'], sec_columns],names=['Variable', 'Model'])
        
sec_columns_R2 = ['SM_R2', 'AR1_R2', 'AR2_R2', 'AR_cay_R2']
multi_columns_R2 = pd.MultiIndex.from_product([['co1', 'co2', 'co3', 'co4'], sec_columns_R2],
                                                  names=['Variable', 'Model'])
        
oos_MSE = pd.DataFrame(index = multi_columns, columns = rows)
oos_R2 = pd.DataFrame(index = multi_columns_R2, columns = rows)

wb_MSE = xlsxwriter.Workbook('Results/OOS_MSE_0709.xlsx')
wb_R2 = xlsxwriter.Workbook('Results/OOS_R2_0709.xlsx')

In [425]:
# worksheet = wb_MSE.add_worksheet('mememe')
# worksheet1 = wb_MSE.add_worksheet('aaa')

In [447]:
writer_MSE = pd.ExcelWriter('results/OOS_MSE_0709.xlsx', engine='xlsxwriter')
writer_R2 = pd.ExcelWriter('results/OOS_R2_0709.xlsx', engine='xlsxwriter')

In [448]:
for i in fun_list:
#     worksheet_MSE = wb_MSE.add_worksheet(i.__name__)
#     worksheet_R2 = wb_R2.add_worksheet(i.__name__)
    
    for j in variables_lag2:
    
        # Prepare X
        X_ = j.join(station_AR2) 

        # Fit models
        x0 = Taylor_init(j.loc[:"1988-01-01"], station_train.loc[:"1988-01-01"], y_train_AR2.loc[:"1988-01-01"], i)

        # Target model
        cls = CLS_Estimator(obj_func = i, x0 = x0, constraints = constraint_func(X_))
        cv_result = cross_validate(cls, X_, y_lag2, cv=cv_outer, scoring = 'neg_mean_squared_error')


        oos_MSE.loc[get_df_name(j)].loc['CLS_MSE'] = -cv_result['test_score']
        print(i.__name__,get_df_name(j))

            ###################################################################################################
        # benchmark model: sm
        sm_pred, sm_mse = bench.sample_mean(y_lag2, "1988-01-01", cv_outer = cv_outer)
        oos_MSE.loc[get_df_name(j)].loc['SM_MSE'] = sm_mse

        #     # benchmark model: Nonlinear
        #     station_n = pd.DataFrame()
        #     x0_n = Taylor_init(X_train, station_n, y_train, i)
        #     nonlinear = CLS_Estimator(obj_func = i, x0 = x0_n, constraints = constraint_func(j))
        #     cv_nonlinear = cross_validate(nonlinear, j, y, cv=cv_outer, scoring = 'neg_mean_squared_error')
        #     nonlinear_mse = cv_nonlinear['test_score']    

        #     # benchmark model: AR1
        #     lr = LR()
        #     cv_nonlinear = cross_validate(lr, df['y_lag'], y, cv=cv_outer, scoring = 'neg_mean_squared_error')

        #     # benchmark model: AR2
        # #     X_AR2 = 
        # #     cv_nonlinear = cross_validate(nonlinear, j, y, cv=cv_outer, scoring = 'neg_mean_squared_error')

        #     # benchmark model: AR+cay
        #     cv_nonlinear = cross_validate(nonlinear, j, y, cv=cv_outer, scoring = 'neg_mean_squared_error')

            ####################################################################################################
        # R2
        R2_sm = 1-(-cv_result['test_score'])/sm_mse
        oos_R2.loc[get_df_name(j)].loc['SM_R2'] = R2_sm
        
        oos_MSE.T.to_excel(writer_MSE, sheet_name=i.__name__)
        oos_R2.T.to_excel(writer_R2, sheet_name=i.__name__)
        
writer_MSE.save()
writer_R2.save()

        # Put into one table
    #     writer_MSE = pd.ExcelWriter('Results/OOS_MSE_0709.xlsx', engine='xlsxwriter')
    #     writer_R2 = pd.ExcelWriter('Results/OOS_R2_0709.xlsx', engine='xlsxwriter')

#         oos_MSE.T.to_excel('Results/oos_MSE_0709.xlsx')
#         oos_R2.T.to_excel('Results/oos_R2_0709.xlsx')

    #     writer_MSE.save()
    #     writer_R2.save()
    #     oos_MSE.T.to_excel(writer_MSE, sheet_name = i.__name__)
    #     oos_R2.T.to_excel(writer_R2, sheet_name = i.__name__)
    #     writer_MSE.save()
    #     writer_R2.save()
    # Export to Excel


sin_func co1
sin_func co2
sin_func co3
sin_func co4
cos_func co1
cos_func co2
cos_func co3
cos_func co4


In [449]:
writer_MSE.close()
writer_R2.close()

In [ ]:
oos_MSE.head(10)

In [ ]:
oos_R2.head(10)

In [433]:
df1 = pd.DataFrame({'Data': [11, 12, 13, 14]})
df2 = pd.DataFrame({'Data': [21, 22, 23, 24]})
df3 = pd.DataFrame({'Data': [31, 32, 33, 34]})

# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('OOS_MSE_0709.xlsx', engine='xlsxwriter')

# Write each dataframe to a different worksheet.
df1.to_excel(writer, sheet_name='Sheet1')
df2.to_excel(writer, sheet_name='Sheet2')
df3.to_excel(writer, sheet_name='Sheet3')

# Close the Pandas Excel writer and output the Excel file.
writer.save()
writer.close()

In [214]:
# for train_index, test_index in cv_outer.split(X):
# #     print(train_index[-3:-1], test_index)
#     print(X.iloc[test_index])

In [ ]:
space = dict()
space['constraints'] = [(), constraint_func(x)]
space['x0'] = [[0.01]*8,[1]*8]

In [ ]:
X = co1.join(station)

In [102]:
constraints = constraint_func(X)

In [115]:
# nls = CLS_Estimator(obj_func = sin_func, x0 = [0.01]*5)
# 
cls = CLS_Estimator(obj_func = sin_func, x0 = [0.01]*5, constraints = constraints)

In [208]:
cv_result = cross_validate(cls, co1.join(station_AR2), y_lag2, cv=cv_outer, scoring = 'neg_mean_squared_error')

In [98]:
# cls.predict(X["1988-01-01":])

In [ ]:
# models, c, model_mse = mr.Nested_CV(X = X, y = y, model = cls, 
#                                              cv_inner = cv_inner, cv_outer = cv_outer, 
#                                              search_method = 'Grid', space = space)

### Benchmark model: sample mean

In [ ]:
sm_pred, sm_mse = bench.sample_mean(y, "1988-01-01", cv_outer = cv_outer)

In [ ]:
sm_mse

### $R^2$ plot

In [ ]:
R2 = 1-(-nls_mse)/sm_mse

In [ ]:
OOS_sin = pd.DataFrame(
    {'nls':-nls_mse, 'sm':sm_mse}
)
OOS_sin.to_excel('OOS_sin.xlsx')

In [ ]:
plt.plot(-nls_mse)
plt.plot(sm_mse)[]

In [ ]:
np.sum(-nls_mse<sm_mse)

In [ ]:
plt.plot(R2)
plt.savefig('sin_oos.jpg')

In [ ]:
R2_new = 1 / (1 + np.exp(-np.array(R2)))
plt.plot(R2_new[0:47])

In [ ]:
# bench.plot_R2(y_test[::3], c, sm_pred, adjust = False, alpha = 0.8)